<a href="https://colab.research.google.com/github/bomb1000/news_keywords_analysis/blob/main/%E6%96%B0%E8%81%9E%E8%AA%9E%E6%96%99%E9%97%9C%E9%8D%B5%E5%AD%97%E5%88%86%E6%9E%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

利用 聯合新聞網 作為主要的文本來源，將範例收集回來的新聞內文存在一個 List 容器變數 `news` 內

In [24]:
import requests
from bs4 import BeautifulSoup

# 利用 requests 對 API 來源發送一個請求
response = requests.get('https://udn.com/news/breaknews/1')
soup = BeautifulSoup(response.text)

In [27]:
news = []
count = 0
for link in soup.find_all('h3', class_='rounded-thumb__title')[:4]:
  news_url = link.a['href']
  news_response = requests.get('https://udn.com' + news_url)
  news_soup = BeautifulSoup(news_response.text)
  news_content = str(news_soup.find('div', class_='article-content__paragraph').text.strip().replace('\n', ' '))
  news.append(news_content)
  count +=1
  print('https://udn.com' + news_url)
  print('a', news_content)
 

https://udn.com/news/story/123450/7188208?from=udn-catehotnews_ch2
 根據《食力》報導，草莓花凋謝後，花梗頂端長著花朵的花托會迅速膨大變紅，成為大家愛吃的草莓，花心內的心皮會結成一粒粒小果鑲嵌在花托上，則稱作「瘦果」。草莓光是一朵花就可以結出許多瘦果，因此草莓表面的小點點其實就是瘦果，卻時常被誤認是草莓的種子，但它其實才是草莓真正的果實。   ▌延伸推薦： 會考自然／考生：計算少、難易適中 考永續發展、綠能和病毒【文教熱話題】  ▪ 整理包／112年國中教育會考 5科完整試題與解答   ▪ 112會考／錯幾題可拿A++？ 五科「容錯題數」一次看   ▪ 會考英語首題與參考題「一模一樣」！ 心測中心宣布送分  ▪ 考生注意！112年學測成績轉換分科測驗級分查詢看這裡  ▪ 會考英文／考生評「歷年最簡單」 題組融合性別議題  ▪ 會考自然／題組跨科融合「中偏易」 素養題考觀念不死記  ▪ 大學剉咧等！ChatGPT代寫報告、學檔 考招受考驗  ▪ 專題／學習歷程，別鬧了！首屆108課綱生心路大公開     112會考 草莓 考題 國中生 自然科
https://udn.com/news/story/7266/7189278?from=udn-catehotnews_ch2
 雲端發票專屬獎有四類獎項，分別是100萬元獎30組、2,000元獎1萬6000組、800元獎10萬組，以及500元獎165萬組，雲端發票專屬獎中獎號碼將公布於財政部稅務入口網站。  今年3-4月期統一發票中獎號碼於今（25）日開獎。 圖／聯合報系資料照片    統一發票 頭獎 電子發票
https://udn.com/news/story/7266/7189014?from=udn-catehotnews_ch2
 彭啟明表示，但目前海水溫度頗高，特別是考慮未來聖嬰現象發展，很讓人憂心未來的發展。「希望它不會走到這一步」，如果真的如某些長期預測，走到強聖嬰，「會是如何呢？實在不敢想像，是老天提醒大家除了淨零更要注意調適防災應變嗎？」中度颱風瑪娃中心位置在關島西北方90公里海面上（鵝鑾鼻東南東方2650公里海面上），以每小時19公里速度，向西北西進行。圖／取自氣象局網站    颱風 彭啟明 氣象局
https://udn.com/new

安裝 `jieba` 斷詞套件與相關工具

In [30]:
# 安裝 jieba 中文斷詞套件
!pip install jieba 

# 下載官方字典檔
!wget https://raw.githubusercontent.com/fxsjy/jieba/master/extra_dict/dict.txt.big


# 載入套件與字典檔
import jieba
jieba.load_userdict("dict.txt.big")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
--2023-05-25 09:36:35--  https://raw.githubusercontent.com/fxsjy/jieba/master/extra_dict/dict.txt.big
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8583143 (8.2M) [text/plain]
Saving to: ‘dict.txt.big’

dict.txt.big        100%[===================>]   8.18M  --.-KB/s    in 0.09s   

2023-05-25 09:36:36 (93.1 MB/s) - ‘dict.txt.big’ saved [8583143/8583143]



Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
DEBUG:jieba:Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.992 seconds.
DEBUG:jieba:Loading model cost 0.992 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.


將每篇新聞文章斷詞後的結果存入 `tokens` 變數，
並且將斷詞後的結果計算成每個單字的出現次數（詞頻），
並存成以「出現單字為 KEY、出現次數為 Value」的 dict 型態變數 `word_count`

In [34]:
# 精確模式斷詞
token_all = []
for d in news:
  token = list(jieba.cut(d, HMM=False)) # token = d 字串斷詞結果
  token_all.extend(token)

word_count = {}

for e in token_all:
  word_count.setdefault(e, 0)
  word_count[e] += 1

print(word_count)
# {'聲明': 4, '主管機關': 3, ...}


{'草莓': 17, '香氣': 1, '十足': 1, '，': 96, '無論': 1, '作': 1, '為': 19, '水果': 1, '單': 1, '吃': 5, '或': 1, '加工': 1, '製成': 1, '甜品': 1, '都': 5, '是': 23, '許多': 3, '大人': 1, '小孩': 1, '的': 52, '最愛': 1, '。': 26, '不過': 1, '你': 2, '夠': 1, '了解': 1, '嗎': 3, '？': 7, '日前': 2, '112': 7, '年': 6, '國': 4, '中': 5, '教育': 2, '會考': 9, '落幕': 1, '有': 10, '考生': 7, '在': 13, '臉': 5, '書': 5, '粉絲': 1, '社團': 1, '「': 41, '學習': 3, '討論': 3, '群': 2, '」': 41, 'PO': 1, '出': 1, '今年': 2, '自然科': 3, '考題': 3, '題': 9, '幹': 3, '詢問': 3, '表面': 4, '如同': 2, '小': 3, '芝麻': 2, '顆粒': 3, '何種': 2, '構造': 5, '不料': 1, '竟': 1, '考倒': 1, '眾': 3, '人': 9, ' ': 136, '\r': 12, '題目': 3, '敘述': 1, '已知': 1, '由': 2, '花托': 4, '處': 1, '膨大': 2, '而來': 1, '*': 2, '記號': 2, '標示': 2, '則': 3, '子房': 4, '發育': 2, '而': 4, '成': 4, '還': 3, '附上': 1, '兩張': 2, '示意圖': 1, '一張': 2, '剖析': 1, '花朵': 2, '以': 4, '該': 2, '究竟': 1, '何': 2, '根據': 2, '心': 2, '測': 3, '中心': 4, '給出': 1, '正解': 1, '答案': 2, '選項': 1, '（': 15, 'C': 2, '）': 15, '果實': 8, '圖': 3, '／': 10, '翻': 1, '攝': 1, '自': 1, '想': 1

將每個單字的出現次數（詞頻）依照出現次數由多到少進行排序


In [42]:
word_count = dict(sorted(word_count.items(), key=lambda item: item[1], reverse = True))

In [43]:
print(word_count)

{' ': 136, '，': 96, '的': 52, '「': 41, '」': 41, '。': 26, '是': 23, '、': 22, '為': 19, '草莓': 17, '（': 15, '）': 15, '日本': 14, '在': 13, '\r': 12, '獎': 11, '有': 10, '／': 10, '台': 10, '會考': 9, '題': 9, '人': 9, '這': 9, '種子': 9, '果實': 8, '我': 8, '▪': 8, '與': 8, '灣': 8, '打包': 8, '？': 7, '112': 7, '考生': 7, '也': 7, '其實': 7, '很': 7, '會': 7, '月': 7, '頭獎': 7, '食物': 7, '年': 6, '不': 6, '上': 6, '組': 6, '4': 6, '日': 6, '萬元': 6, '1': 6, '元': 6, '公里': 6, '吃': 5, '都': 5, '中': 5, '臉': 5, '書': 5, '構造': 5, '了': 5, '就': 5, '！': 5, '3': 5, '統一發票': 5, '末': 5, '碼': 5, '相同': 5, '；': 5, '文化': 5, '國': 4, '表面': 4, '花托': 4, '子房': 4, '而': 4, '成': 4, '以': 4, '中心': 4, '他': 4, '它': 4, '到': 4, '5': 4, '最': 4, '-': 4, '期': 4, '中獎號碼': 4, ',': 4, '000': 4, '萬': 4, '颱風': 4, '海面': 4, '彭': 4, '啟': 4, '明': 4, '歐': 4, '吉': 4, '桑': 4, '服務': 4, '喜歡': 4, '許多': 3, '嗎': 3, '學習': 3, '討論': 3, '自然科': 3, '考題': 3, '幹': 3, '詢問': 3, '小': 3, '顆粒': 3, '眾': 3, '題目': 3, '則': 3, '還': 3, '測': 3, '圖': 3, '不是': 3, '指': 3, '著': 3, '認': 3, '東西': 3, '但': 3